In [1]:
import pandas as pd
import numpy as np
from id_nodes import id_nodes
import id_changes
import update_bom
import importlib
import format_xlsx

In [2]:
master_bom_path = r"..\ShortMaster-0015442-FS-8100-BOM-20200326.xlsx"
new_bom_path = r"..\ShortTest-0015442-FS-8100-BOM-20200326.xlsx"

In [3]:
master_bom_df = pd.read_excel(master_bom_path)
new_bom_df = pd.read_excel(new_bom_path)

In [4]:
master_bom_df.index.set_names('Order', inplace=True)
new_bom_df.index.set_names('Order', inplace=True)

In [5]:
master_bom_df['Node ID'] = id_nodes(master_bom_df)
new_bom_df['Node ID'] = id_nodes(new_bom_df)

In [6]:
master_bom_df.set_index('Node ID', append=True, inplace=True)
new_bom_df.set_index('Node ID', append=True, inplace=True)

In [7]:
main_widths = [5, 60, 60, 60, 15, 60, 10, 10, 15]
key_width = 23

In [28]:
master_bom_df

,,Level,File Name,Name,D_TITLE,D_TITLE2,D_DESCRIPTION,Quantity,Version,State,SupportColumn1,SupportColumn2,SupportColumn3,SupportColumn4,SupportColumn5,SupportColumn6
Order,Node ID,,,,,,,,,,,,,,,
0,->0015442.asm,0,0015442.asm,0015442.asm,"SHIPPING CONFIG, GS2",-,"SHIPPING CONFIG, GS2",NaN,0-00.50,Processing,NaN,NaN,NaN,NaN,NaN,NaN
1,0015442.asm->0019043.prt,1,0019043.prt,FS-8100 ID SURFACE MODEL,FS-8100 ID SURFACE MODEL,-,FS-8100 ID SURFACE MODEL,1.0,0-00.37,Processing,NaN,NaN,NaN,NaN,NaN,NaN
2,0015442.asm->0018435.asm,1,0018435.asm,FS-8100 Chassis assembly - full,FS-8100 Chassis assembly - full,NaN,FS-8100 Chassis assembly - full,1.0,0-00.24,Processing,NaN,NaN,NaN,NaN,NaN,NaN
3,0015442.asm->0018435.asm->0018442.asm,2,0018442.asm,FS-8100 Chassis Base Assembly,FS-8100 Chassis Base Assembly,NaN,FS-8100 Chassis Base Assembly,1.0,0-00.6,Processing,NaN,NaN,NaN,NaN,NaN,NaN
4,0015442.asm->0018435.asm->0018442.asm->0018441.prt,3,0018441.prt,FS-9100 base blate - btm plate,FS-9100 base blate - btm plate,NaN,FS-9100 base blate - btm plate,1.0,0-00.7,Processing,NaN,NaN,NaN,NaN,NaN,NaN
5,0015442.asm->0018435.asm->0018442.asm->0018541.prt,3,0018541.prt,FS-8100 Chassis base front edge,FS-8100 Chassis base front edge,NaN,FS-8100 Chassis base front edge,1.0,0-00.1,Processing,NaN,NaN,NaN,NaN,NaN,NaN
6,0015442.asm->0018435.asm->0018442.asm->0011799.prt,3,0011799.prt,"STUD, SC-FH, M6X1.0, 18MM LG, 1.6MM MIN THK, S...","STUD,SELF-CLINCHING,THD,M6-18,FHS",NaN,"STUD,SELF-CLINCHING,THD,M6-18,FHS",12.0,0-00.4,Alpha Prototype,NaN,NaN,NaN,NaN,NaN,NaN
7,0015442.asm->0018435.asm->0018442.asm->0014161.prt,3,0014161.prt,"3 PROJECTION WELDNUT, 3/4""-10","3 PROJECTION WELDNUT, 3/4""-10",NaN,"3 PROJECTION WELDNUT, 3/4""-10",4.0,0-00.1,Released,NaN,NaN,NaN,NaN,NaN,NaN
8,0015442.asm->0018435.asm->0018442.asm->0012468.prt,3,0012468.prt,RIVET,RIVET,NaN,RIVET,4.0,0-00.2,Released,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
new_bom_df

,,Level,File Name,Name,D_TITLE,D_TITLE2,D_DESCRIPTION,Quantity,Version,State,AddedColumn1
Order,Node ID,,,,,,,,,,
0,->0015442.asm,0,0015442.asm,0015442.asm,"SHIPPING CONFIG, GS2",-,"SHIPPING CONFIG, GS2",NaN,0-00.50,Processing,NaN
1,0015442.asm->0019043.prt,1,0019043.prt,FS-8100 ID SURFACE MODEL,FS-8100 ID SURFACE MODEL,-,FS-8100 ID SURFACE MODEL,1.0,0-00.37,Processing,NaN
2,0015442.asm->0018435.asm,1,0018435.asm,FS-8100 Chassis assembly - full,FS-8100 Chassis assembly - full,NaN,FS-8100 Chassis assembly - full,1.0,0-00.24,Processing,NaN
3,0015442.asm->0018435.asm->0018442.asm,2,0018442.asm,FS-8100 Chassis Base Assembly,FS-8100 Chassis Base Assembly,NaN,FS-8100 Chassis Base Assembly,1.0,0-00.6,Processing,NaN
4,0015442.asm->0018435.asm->0018442.asm->0018441.prt,3,0018441.prt,FS-9100 base blate - btm plate,FS-9100 base blate - btm plate,NaN,FS-9100 base blate - btm plate,1.0,0-00.7,Processing,NaN
5,0015442.asm->0018435.asm->0018442.asm->0011799.prt,3,0011799.prt,"STUD, SC-FH, M6X1.0, 18MM LG, 1.6MM MIN THK, S...","STUD,SELF-CLINCHING,THD,M6-18,FHS",NaN,"STUD,SELF-CLINCHING,THD,M6-18,FHS",12.0,0-00.4,Alpha Prototype,NaN
6,0015442.asm->0018435.asm->0018442.asm->0014161.prt,3,0014161.prt,"3 PROJECTION WELDNUT, 3/4""-10","3 PROJECTION WELDNUT, 3/4""-10",NaN,"3 PROJECTION WELDNUT, 3/4""-10",4.0,0-00.1,Released,NaN
7,0015442.asm->0018435.asm->0018442.asm->0012468.prt,3,0012468.prt,RIVET,RIVET,NaN,RIVET,4.0,0-00.2,Released,NaN
8,0015442.asm->0018435.asm->0018436.prt,2,0018436.prt,FS-8100 Chassis vert member SM-Box,FS-8100 Chassis vert member SM-Box,NaN,FS-8100 Chassis vert member SM-Box,4.0,0-00.15,Processing,NaN


In [8]:
deleted_nodes = id_changes.get_deleted_nodes(master_bom_df, new_bom_df)
added_nodes = id_changes.get_added_nodes(master_bom_df, new_bom_df)
reordered_nodes = id_changes.get_reordered_nodes(master_bom_df, new_bom_df)

In [9]:
added_columns = id_changes.get_added_columns(master_bom_df, new_bom_df)

In [10]:
updated_elements = id_changes.get_updated_elements(master_bom_df, new_bom_df, order_only=None)

In [11]:
updated_nodes = id_changes.get_updated_nodes(master_bom_df, new_bom_df, order_only=None)

In [12]:
updated_bom_df = update_bom.merge_support_columns(master_bom_df, new_bom_df)

In [13]:
updated_bom_df.index.set_names(['Order', 'Node ID'], inplace=True)

In [14]:
writer = pd.ExcelWriter('pandasEx.xlsx', engine ='xlsxwriter')

In [15]:
updated_bom_df.to_excel(writer, sheet_name ='Updated BOM', index=False)
master_bom_df.loc[(slice(None), deleted_nodes), :].to_excel(writer, sheet_name ='Removed Nodes', index=False)
new_bom_df.loc[(slice(None), added_nodes), :].to_excel(writer, sheet_name ='Added Nodes', index=False)
master_bom_df.loc[(slice(None), updated_nodes), :].to_excel(writer, sheet_name ='Updated Nodes', index=False)
master_bom_df.loc[(slice(None), reordered_nodes), :].to_excel(writer, sheet_name ='Reordered Nodes', index=False)

In [16]:
workbook = writer.book

In [17]:
worksheet_names = [worksheet.name for worksheet in workbook.worksheets()]

In [18]:
added_column_header_format = workbook.add_format({'bg_color': '#ADD8E6', 'bold': True, 'border': 1, 'align': 'center_across', 'valign': 'top'})
added_node_format = workbook.add_format({'bg_color': '#00B0F0'})
reordered_node_format = workbook.add_format({'bg_color': '#92D050'})
updated_element_format = workbook.add_format({'bg_color': '#FFFF00'})

In [19]:
updated_bom_sheet = workbook.get_worksheet_by_name('Updated BOM')

In [20]:
column_map = {column_name: column_position for column_position, column_name in enumerate(updated_bom_df.columns)}

In [21]:
for column_name in added_columns:
    updated_bom_sheet.write(0, updated_bom_df.columns.get_loc(column_name), column_name, added_column_header_format)

In [22]:
for node_position, node_name in enumerate(updated_bom_df.index.get_level_values('Node ID')):
    if node_name in added_nodes:
        updated_bom_sheet.write(node_position + 1, updated_bom_df.columns.get_loc('Level'), updated_bom_df.loc[(slice(None), node_name), 'Level'][0], added_node_format)
        updated_bom_sheet.write_string(node_position + 1, updated_bom_df.columns.get_loc('File Name'), updated_bom_df.loc[(slice(None), node_name), 'File Name'][0], added_node_format)

    if node_name in reordered_nodes:
        updated_bom_sheet.write(node_position + 1, updated_bom_df.columns.get_loc('Level'), updated_bom_df.loc[(slice(None), node_name), 'Level'][0], reordered_node_format)
        updated_bom_sheet.write_string(node_position + 1, updated_bom_df.columns.get_loc('File Name'), updated_bom_df.loc[(slice(None), node_name), 'File Name'][0], reordered_node_format)

    if node_name in updated_nodes:
        for updated_column in updated_elements.loc[node_name, updated_elements.loc[node_name]].index.values:
            updated_bom_sheet.write(node_position + 1, updated_bom_df.columns.get_loc(updated_column), updated_bom_df.loc[(slice(None), node_name), updated_column][0], updated_element_format)

In [23]:
workbook.add_worksheet('Key')

In [24]:
key_sheet = workbook.get_worksheet_by_name('Key')
key_sheet.write(0, 0, 'Re-ordered nodes', reordered_node_format)
key_sheet.write(1, 0, 'Added nodes or columns', added_node_format)
key_sheet.write(2, 0, 'Updated nodes', updated_element_format)
key_sheet.set_column(0, 0, key_width)

0

In [25]:
max_column_widths = format_xlsx.get_max_column_widths(updated_bom_df, buffer=2)

In [27]:
writer.save()